In [1]:
import sys
sys.path.insert(1, '/Users/brian/crypto/crocket')

from os import environ
from os.path import join

from crocket.sql.sql import Database
from crocket.utilities.credentials import get_credentials

In [2]:
# Input parameters

In [3]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = '173.255.241.200'
DATABASE_NAME = 'BITTREX2'

In [4]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

# Create database connection
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [6]:
data = db.get_all_tables()

AttributeError: 'NoneType' object has no attribute 'cursor'

In [ ]:
# Get data from SQL database

# Get initial coine data

# Get data every minute